In [4]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import re

In [5]:
company_name = "Kebo"
columns = ['companyName', 'datePublished', 'ratingValue', 'reviewBody']

all_reviews_df = pd.DataFrame(columns=columns)

response = requests.get("https://ca.trustpilot.com/review/kabo.co")
soup = BeautifulSoup(response.text)
item = soup.find(
    name='p',
    attrs={'class':'typography_body-l__KUYFJ typography_appearance-default__AAY17'}
)
N = int(item.contents[0].replace(',',''))
N

862

In [6]:
page = "https://ca.trustpilot.com/review/kabo.co"
while True:
    print("Scraping from page " + page)
    
    response = requests.get(page)
    soup = BeautifulSoup(response.text)
    reviews = soup.find(
        name='script',
        attrs={'id':"__NEXT_DATA__", 'type':"application/json"}
    )
    content = reviews.contents[0]
    result = json.loads(content)
    reviews = result['props']['pageProps']['reviews']
    for review in reviews:
        review_row = {}
        review_row['companyName'] = [company_name]
        review_row['datePublished'] = [review['dates']['publishedDate']]
        review_row['ratingValue'] = [review['rating']]
        review_row['reviewBody'] = [review['text']]
        review_df = pd.DataFrame(review_row)
        all_reviews_df = pd.concat([all_reviews_df, review_df], ignore_index = True)
        all_reviews_df.reset_index()
    next_page = soup.find_all(name="a", string='Next page')[0]
    if 'href' not in str(next_page):
        break
    page = 'https://ca.trustpilot.com' + soup.find_all(name="a", string='Next page')[0]['href']

Scraping from page https://ca.trustpilot.com/review/kabo.co
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=2
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=3
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=4
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=5
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=6
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=7
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=8
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=9
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=10
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=11
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=12
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=13
Scraping from page https://ca.trustpilot.com/review/kabo.co?page=14
Scraping from page https://ca.trustpilot.com/review/kabo.co?page

In [7]:
all_reviews_df

,companyName,datePublished,ratingValue,reviewBody
0,Kebo,2024-08-25T19:41:09.000Z,5,I moved to Canada with my dog for six months a...
1,Kebo,2024-06-16T18:32:37.000Z,1,Less than zero! Where is my order? Horrible cu...
2,Kebo,2022-07-29T11:59:29.000Z,5,We adopted Maisey in October of 2021. She is a...
3,Kebo,2022-11-22T05:43:52.000Z,5,Charlie had pancreatitis and we were told to g...
4,Kebo,2022-11-16T00:00:28.000Z,5,As an owner of a Frenchie with allergies that ...
...,...,...,...,...
751,Kebo,2019-06-13T13:36:35.000Z,5,My guy Winston loves his Kabo. We had got him ...
752,Kebo,2019-06-13T12:47:40.000Z,5,Mealtime has always been popular but I simply ...
753,Kebo,2019-06-13T12:45:21.000Z,5,Big Kabo Fan! I have a 6 year old rescue and n...
754,Kebo,2019-06-13T00:34:05.000Z,5,"Our dog Leo loves the Kabo food, he licks the ..."


In [8]:
all_reviews_df.to_csv('Kabo reviews.csv')